In [ ]:
!pip install wandb

In [ ]:
# mecab 설치
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [2]:
#!pip install kogpt2_transformers
!pip install rouge
!pip install pytorch-lightning==1.1.0
!pip install streamlit==0.72.0

     |████████████████████████████████| 665 kB 6.3 MB/s            
     |████████████████████████████████| 7.4 MB 6.0 MB/s            
     |████████████████████████████████| 812 kB 60.3 MB/s            
     |████████████████████████████████| 78 kB 11.7 MB/s            
     |████████████████████████████████| 182 kB 82.7 MB/s            


     |████████████████████████████████| 4.7 MB 73.4 MB/s eta 0:00:01

     |████████████████████████████████| 4.7 MB 73.4 MB/s            
     |████████████████████████████████| 55 kB 8.0 MB/s             
     |████████████████████████████████| 62 kB 2.5 MB/s             
     |████████████████████████████████| 340 kB 74.5 MB/s            


In [1]:
%cd /aiffel/aiffel/aiffelthon/KoGPT2-summarization/

/aiffel/aiffel/aiffelthon/KoGPT2-summarization


In [ ]:
# 라이브러리 불러오기
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers
import torch
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import Dataset
from kogpt2_transformers import get_kogpt2_model, get_kogpt2_tokenizer
from soft_embedding import SoftEmbedding
import csv
import pandas as pd
from datasets import Dataset
from rouge import Rouge
import re
import os

In [12]:
# 라이브러리 불러오기

import argparse
import logging
import os
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
import torch.nn.functional as F
from pytorch_lightning import loggers as pl_loggers
from torch.utils.data import DataLoader, Dataset
import numpy as np
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
from dataset_summary import KoGPTSummaryDataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from soft_embedding import SoftEmbedding
import pdb

In [ ]:
# Category별 DataFrame을 출력 (train)
train = pd.read_csv('/aiffel/aiffel/aiffelthon/csv/train.tsv', delimiter = '\t')
Category = train['Category'].unique()
data_path = '/aiffel/aiffel/aiffelthon/csv/'
def train_categori_ext(data):
    df = pd.DataFrame()
    for c in Category:
        df = pd.concat([df, data[data['Category'] == c].iloc[0:int(len(data[data['Category'] == c])*0.2)]], axis = 0)
        df.reset_index(inplace=True, drop=True)
        df.to_csv(data_path+'train_20%.tsv', sep = '\t', encoding = 'utf-8', index = False)
    return df

In [ ]:
# train_categori_ext(train)

In [ ]:
# Category별 DataFrame을 출력
val = pd.read_csv('/aiffel/aiffel/aiffelthon/csv/val.tsv', delimiter = '\t')
Category = val['Category'].unique()
data_path = '/aiffel/aiffel/aiffelthon/csv/'
def val_categori_ext(data):
    df = pd.DataFrame()
    for c in Category:
        df = pd.concat([df, data[data['Category'] == c].iloc[0:int(len(data[data['Category'] == c])*0.2)]], axis = 0)
        df.reset_index(inplace=True, drop=True)
        df.to_csv(data_path+'val_20%.tsv', sep = '\t', encoding = 'utf-8', index = False)
    return df

In [8]:
import pandas as pd

In [17]:
test = pd.read_csv('/aiffel/aiffel/aiffelthon/KoGPT2-summarization/data/test.tsv', delimiter = '\t')

In [18]:
test

,Id,Text,Summary,Category
0,9effd65e-e18f-5510-8866-d6a677314c9c,"['웅', '영업팀과장님이 보내줬는데 팀장님이 해줄지 모르겠다 저번에 부산갈때도 숙...",팀장님이 출장 가서 머물 숙소를 계속해서 더 싼 데로 하게 한다고 이야기하고 있다.,일과 직업
1,49b9f9b7-a79d-5283-9553-95b8f156ff90,"['너는 잘가라....회사.... 선택 잘해..', '알겠어 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 많...",이제 이력서를 쓰고 영어도 해야 한다고 해서 첫 회사를 잘 들어가라고 했다.,일과 직업
2,df6b6bac-648b-5d32-86a3-9b8b215e5f95,"['느낌상 대통령까지는 아니고 오시면 여사님정도오시지않을까 ㅋㅋㅋ', '그러면서',...",느낌상 대통령까지는 아니고 오시면 여사님 정도 오시지 않을까라며 이에 대해 이야기하...,일과 직업
3,3043293f-7bb5-5edb-9b1f-85120e305c3d,"['숨만수이ㅓ도 숨만쉬어도 100 이내', '한달안에 일 무조건 해야대', '아 딱...",한 달 안에 무조건 일을 시작해서 돈을 벌어야 한다.,일과 직업
4,2b9b9c39-2721-50e6-af3e-e5c00138f43d,"['목요일은 외근이구 금요일은 출장!!!', '금요일이 당진이양?', '아닝아닝 1...",목요일에 외근이고 금요일에 출장인데 당진은 10일에 간다.,일과 직업
...,...,...,...,...
1741,b03b9ad0-62fa-54a5-b552-69049bd642ee,"['엄마 점심식사 뭐먹었어요??', '우린 중국집 시켜먹었다', '오오 짜장면 탕수...","엄마에게 오늘 점심 뭐 먹었냐고 묻고 엄마가 중국집에서 볶음밥, 잡채밥 시켜 먹었다...",식음료
1742,ec83f40a-3c01-581d-962c-80fd62a5c888,"['매운거', '담에 점심 엽떡', '자신있음', '엽떡 먹는 사람', '오 저색기...",엽떡(엽기 떡볶이) 제일 매운맛을 3명에서 도전하기로 했다.,식음료
1743,2b8dad32-cb6a-5d2a-89c4-06ba5f8dc8f8,"['아 술마시고싶다 취할줄알았는데 ㅋㅋㅋㅋㅋㅋ', '누룩오자 누룩 군고구마막걸리 존...",군고구마막걸리가 맛있다고 하니 빨리 마셔보고 싶다고 한다.,식음료
1744,bd803ef5-c166-54ad-8d96-ac73fb055b3d,"['종촌동에서 뭘 주워먹어야겠다... 어른들 저녁먹고 가신대 뭐사먹지,.....',...",어른들이 저녁 먹고 가신다고 해서 뭘 사 먹을까 하니 근처에 국수나무가 맛있다고 한다.,식음료


In [ ]:
# val_categori_ext(val)

In [90]:
# torch cash 초기화
import gc
gc.collect()
torch.cuda.empty_cache()

Model : KoGPT-2

Parametors size : 

[How to use GPT2LMHeadModel for conditional generation](https://github.com/huggingface/transformers/issues/970)

Data NAME : Ai_hub 대화문 요약 Data Set

In [4]:
# 데이터 전처리

def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열 (.) 제거
    sentence = re.sub(r'[ㄱ-ㅎㅏ-ㅣ.,?!]+[/ㄱ-ㅎㅏ-ㅣ.,?!]', '', sentence) # 여러개 자음, 모음, 구두점에서 마지막 1개만 남긴다
    sentence = re.sub("[^가-힣a-z0-9-.,!?]", " ", sentence) # 지정한 문자 제외 공백으로 전환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거

    return sentence

In [5]:
# 데이터 로드 및 전처리와 메타데이터 적용 

def data_load(data_path = '/aiffel/aiffel/aiffelthon/csv/train_5%.tsv'):
    data = []
    train_df = pd.read_csv(data_path, delimiter = '\t')
    #train_df = train_df.iloc[::2, :] # 짝수 row data만 추출한다.
    #train_df.reset_index(drop=True, inplace = True) # 인덱스 초기화
    
    # 전처리 적용
    clean_text = []
    
    for s in tqdm(train_df['Text']):
        clean_text.append(preprocess_sentence(s))
                          
    train_df['Text'] = clean_text
    
    clean_headlines = []
                          
    for s in tqdm(train_df['Summary']):
        clean_headlines.append(preprocess_sentence(s))
        
    train_df['Summary'] = clean_headlines 
    
#     # 메타데이터 적용
#     def metadata_train(train_df): 
#         meta = '#'+train_df['Category']+'# '
#         train_df['Text'] = meta + train_df['Text']
#         return train_df['Text']
    
#     metadata_train(train_df)
    
    # Series > list
    train_df['Text'].tolist()
    train_df['Summary'].tolist()
    
    return train_df['Text'], train_df['Summary']

def val_load(data_path = '/aiffel/aiffel/aiffelthon/csv/val_5%.tsv'):
    data = []
    val_df = pd.read_csv(data_path, delimiter = '\t')
    #val_df = val_df.iloc[::2, :]
    #val_df.reset_index(drop=True, inplace = True)
    

    # 전처리 적용
    clean_text_val = []
                          
    for s in tqdm(val_df['Text']):
        clean_text_val.append(preprocess_sentence(s))
    
    val_df['Text'] = clean_text_val
    
    clean_headlines_val = []
                          
    for s in tqdm(val_df['Summary']):
        clean_headlines_val.append(preprocess_sentence(s))
        
    val_df['Summary'] = clean_headlines_val 
    
    # 메타데이터 적용
#     def metadata_val(val_df): 
#         meta = '#'+val_df['Category']+'# '
#         val_df['Text'] = meta + val_df['Text']
#         return val_df['Text']
    
#     metadata_val(val_df)
                          
    # Series > list
    val_df['Text'].tolist()
    val_df['Summary'].tolist()
    
    return val_df['Text'], val_df['Summary']

In [ ]:
data_1 = data_load()

In [ ]:
data_1

In [ ]:
data_2 = val_load()

In [ ]:
data_2

### hyparam은 Code를 가져올 때 설정을 유지중입니다. GPT-2  학습이 제대로 이루어진게 어제부터라서 수정하지는 못했습니다.

### 그래도 수정을 한 부분은 문장을 생성할 때 eos_token, pad_token이 생성되어 이 부분을 출력하지 않도록 bad_words_index를 사용했습니다.

In [4]:
# # Soft_embedding 제작

# import torch
# import torch.nn as nn

# # Soft_embedding Class
# class SoftEmbedding(nn.Module):
#     def __init__(self, wte : nn.Embedding, prompt_length : int = 20, random_range : float = 0.5, initialize_from_vocab: bool = True):
#         """appends learned embedding to 
#         Args:
#             wte (nn.Embedding): original transformer word embedding (기존 트랜스포머의 word embedding)
#             prompt_length (int, optional): number of tokens for task. Defaults to 10. (테스크 토큰 수, 기본 값 10개)
#             random_range (float, optional): range to init embedding (if not initialize from vocab). Defaults to 0.5.
#             (만약 vocab에서 초기화하지 않으면 기본 값 0.5)
#             initialize_from_vocab (bool, optional): initalizes from default vocab. Defaults to True. (기본 vocab에서 초기화)
#         """
#         super(SoftEmbedding, self).__init__()
#         self.wte = wte
#         slef.prompt_length = prompt_length
#         self.learned_embedding = nn.parameter.Parameter(self.initialize_embedding(
#         wte, prompt_length, random_range, initialize_from_vocab))
        
#     def initialize_embedding(self, wte : nn.Embedding, prompt_length : int = 10, random_range : float = 0.5, initialize_from_vocab: bool = True)
#         """initializes learned embedding
#         Args:
#             same as __init__
#         Returns:
#             torch.float: initialized using original schemes
#         """
#         if initialize_from_vocab:
#             return self.wte.weight[:prompt_length].clone().detach()
#         return torch.FloatTensor(prompt_length, wte.weight.size(1)).uniform_(-random_range, random_range)
    
#     def forward(self, tokens)

SyntaxError: invalid syntax (2682299943.py, line 23)

In [5]:
train_set = KoGPTSummaryDataset(file = '/aiffel/aiffel/aiffelthon/csv/train_5%.tsv', tok = get_kogpt2_tokenizer(), max_len = 256)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [6]:
val_set = KoGPTSummaryDataset(file = '/aiffel/aiffel/aiffelthon/csv/val_5%.tsv', tok = get_kogpt2_tokenizer(), max_len = 256)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [3]:
# dm 제작

parser = argparse.ArgumentParser(description='KoGPT2 Summarization')

parser.add_argument('--checkpoint_path',
                    type=str,
                    help='checkpoint path')

class ArgsBase():
    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = argparse.ArgumentParser(
            parents=[parent_parser], add_help=False)
        parser.add_argument('--train_file',
                            type=str,
                            default='data/train.tsv',
                            help='train file')

        parser.add_argument('--test_file',
                            type=str,
                            default='data/test.tsv',
                            help='test file')

        parser.add_argument('--batch_size',
                            type=int,
                            default=4,
                            help='')
        parser.add_argument('--max_len',
                            type=int,
                            default=256,
                            help='max seq len')
        return parser

logger = logging.getLogger()
logger.setLevel(logging.INFO)
        
class AbstractiveKoGPT2(pl.LightningDataModule):
    def __init__(self, train_file,
                 test_file, tok,
                 max_len=256,
                 batch_size=4,
                 num_workers=4,
                 prompt_length=0):
        super().__init__()
        self.batch_size = batch_size
        self.max_len = max_len
        self.train_file_path = train_file
        self.test_file_path = test_file
        self.prompt_length = prompt_length
        if tok is None:
            self.tok = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                       bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                       pad_token='<pad>', mask_token='<mask>') 
        else:
            self.tok = tok  
        self.num_workers = num_workers
            
    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = argparse.ArgumentParser(
            parents=[parent_parser], add_help=False)
        parser.add_argument('--num_workers',
                            type=int,
                            default=5,
                            help='num of worker for dataloader')
        return parser
    
    def setup(self, stage):
        # split dataset
        self.train = KoGPTSummaryDataset(file=self.train_file_path,
                                 tok=self.tok,
                                 max_len=self.max_len,
                                 prompt_length=self.prompt_length)
        
        self.test = KoGPTSummaryDataset(file=self.test_file_path,
                                tok=self.tok,
                                max_len=self.max_len,
                                prompt_length=self.prompt_length)

    def train_dataloader(self):
        train = DataLoader(self.train,
                           batch_size=self.batch_size,
                           num_workers=self.num_workers, shuffle=True)
        return train

    def val_dataloader(self):
        val = DataLoader(self.test,
                         batch_size=self.batch_size,
                         num_workers=self.num_workers, shuffle=False)
        return val

    def test_dataloader(self):
        test = DataLoader(self.test,
                          batch_size=self.batch_size,
                          num_workers=self.num_workers, shuffle=False)
        return test
            
class Base(pl.LightningModule):
    def __init__(self, hparams, **kwargs) -> None:
        super(Base, self).__init__()
        self.save_hyperparameters(hparams)
        
    @staticmethod
    def add_model_specific_args(parent_parser):
        # add model specific args
        parser = argparse.ArgumentParser(
            parents=[parent_parser], add_help=False)

        parser.add_argument('--lr',
                            type=float,
                            default=3e-5,
                            help='The initial learning rate')

        parser.add_argument('--warmup_ratio',
                            type=float,
                            default=0.1,
                            help='warmup ratio')

        parser.add_argument('--model_path',
                            type=str,
                            default=None,
                            help='kobart model path')
        
        parser.add_argument('--prompt_length',
                            type=int,
                            default=20,
                            help='number of trainable parameters')
        
        return parser
       
    def configure_optimizers(self):
        
        prompt_parameters = self.model.get_input_embeddings().learned_embedding
        optimizer = AdamW([prompt_parameters],
                          lr=self.hparams.lr, correct_bias=False)
        # warm up lr
        num_workers = self.hparams.num_workers
        data_len = len(self.train_dataloader().dataset)
        logging.info(f'number of workers {num_workers}, data length {data_len}')
        num_train_steps = int(data_len / (self.hparams.batch_size * num_workers) * self.hparams.max_epochs)
        logging.info(f'num_train_steps : {num_train_steps}')
        num_warmup_steps = int(num_train_steps * self.hparams.warmup_ratio)
        logging.info(f'num_warmup_steps : {num_warmup_steps}')
        scheduler = get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=num_warmup_steps, num_training_steps=num_train_steps)
        lr_scheduler = {'scheduler': scheduler, 
                        'monitor': 'loss', 'interval': 'step',
                        'frequency': 1}
        return [optimizer], [lr_scheduler]

In [4]:
# Model

class KoGPTConditionalGeneration(Base):
    def __init__(self, hparams, **kwargs):
        super(KoGPTConditionalGeneration, self).__init__(hparams, **kwargs)
        self.model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
        
        for p in self.model.parameters():
            p.requires_grad = False
            
        self.update_embedding()
        self.pad_token_id = 0
        self.tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                       bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                       pad_token='<pad>', mask_token='<mask>')
        
        self.neg = 0
        self.loss_function = torch.nn.CrossEntropyLoss(reduction='none')
        
    def update_embedding(self):
        s_wte = SoftEmbedding(self.model.get_input_embeddings(), 
                      prompt_length=self.hparams.prompt_length, 
                      initialize_from_vocab=True)
        self.model.set_input_embeddings(s_wte)
        
    def forward(self, inputs):
        output = self.model(inputs, return_dict=True)
        return output.logits    
        
    def training_step(self, batch, batch_idx):
        token_ids = batch['input']
        mask = batch['mask']
        label = batch['label']

        out = self(token_ids)        
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, self.neg * torch.ones_like(out))
        loss = self.loss_function(mask_out.transpose(2, 1), label)
        loss_avg = loss.sum() / mask.sum()
        self.log('train_loss', loss_avg)
        return loss_avg
        
    def validation_step(self, batch, batch_idx):
        token_ids = batch['input']
        mask = batch['mask']
        label = batch['label']
        
        out = self(token_ids)
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, self.neg * torch.ones_like(out))
        loss = self.loss_function(mask_out.transpose(2, 1), label)
        loss_avg = loss.sum() / mask.sum()
        return (loss_avg)

    def validation_epoch_end(self, outputs):
        losses = []
        for loss in outputs:
            losses.append(loss)
        self.log('val_loss', torch.stack(losses).mean(), prog_bar=True)    

# 오류 해결

[parse 오류 해결]https://uding.tistory.com/49

[TypeError: expected str, bytes or os.PathLike object, not NoneType]
- checkpoint_path 연결

[MisconfigurationException: No `train_dataloader()` method defined. Lightning `Trainer` expects as minimum a `training_step()`, `train_dataloader()` and `configure_optimizers()` to be defined]()

In [5]:
# train

if __name__ == '__main__':
    parser = Base.add_model_specific_args(parser)
    parser = ArgsBase.add_model_specific_args(parser)
    parser = AbstractiveKoGPT2.add_model_specific_args(parser)
    parser = pl.Trainer.add_argparse_args(parser)
    args,_ = parser.parse_known_args()
    logging.info(args)
    
    checkpoint_path = "/aiffel/aiffel/aiffelthon/checkpoint_4"
    save_ckpt_path = f"{checkpoint_path}/kogpt2-abstractive_1epoch.pth"
    
    model = KoGPTConditionalGeneration(args)
    
    dm = AbstractiveKoGPT2(args.train_file,
                        args.test_file,
                        None,
                        batch_size=args.batch_size,
                        max_len=args.max_len,
                        num_workers=args.num_workers,
                        prompt_length=args.prompt_length)
    
    checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor='val_loss',
                                                       dirpath=checkpoint_path,
                                                       filename='model_chp/{epoch:02d}-{val_loss:.3f}',
                                                       verbose=True,
                                                       save_last=True,
                                                       mode='min',
                                                       save_top_k=3)
    tb_logger = pl_loggers.TensorBoardLogger(os.path.join(checkpoint_path, 'tb_logs'))
    lr_logger = pl.callbacks.LearningRateMonitor()
    trainer = pl.Trainer.from_argparse_args(args, logger=tb_logger,
                                            callbacks=[checkpoint_callback, lr_logger])
    
    trainer.fit(model, dm)

INFO:root:Namespace(checkpoint_path=None, lr=3e-05, warmup_ratio=0.1, model_path=None, prompt_length=20, train_file='data/train.tsv', test_file='data/test.tsv', batch_size=4, max_len=256, num_workers=5, logger=True, checkpoint_callback=True, default_root_dir=None, gradient_clip_val=0, process_position=0, num_nodes=1, num_processes=1, gpus=<function _gpus_arg_default at 0x7f91379f13a0>, auto_select_gpus=False, tpu_cores=<function _gpus_arg_default at 0x7f91379f13a0>, log_gpu_memory=None, progress_bar_refresh_rate=1, overfit_batches=0.0, track_grad_norm=-1, check_val_every_n_epoch=1, fast_dev_run=False, accumulate_grad_batches=1, max_epochs=1000, min_epochs=1, max_steps=None, min_steps=None, limit_train_batches=1.0, limit_val_batches=1.0, limit_test_batches=1.0, val_check_interval=1.0, flush_logs_every_n_steps=100, log_every_n_steps=50, accelerator=None, sync_batchnorm=False, precision=32, weights_summary='top', weights_save_path=None, num_sanity_val_steps=2, truncated_bptt_steps=None, r

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
Saving latest checkpoint...
INFO:lightning:Saving latest checkpoint...


MisconfigurationException: ModelCheckpoint(monitor='val_loss') not found in the returned metrics: ['train_loss']. HINT: Did you call self.log('val_loss', tensor) in the LightningModule?

In [2]:
!python train_ptuning.py  --gradient_clip_val 1.0 --max_epochs 1 --default_root_dir logs  --gpus 1 --batch_size 4 --num_workers 4 --max_len 256

INFO:root:Namespace(checkpoint_path=None, lr=3e-05, warmup_ratio=0.1, model_path=None, prompt_length=20, train_file='data/train.tsv', test_file='data/test.tsv', batch_size=4, max_len=258, num_workers=4, logger=True, checkpoint_callback=True, default_root_dir='logs', gradient_clip_val=1.0, process_position=0, num_nodes=1, num_processes=1, gpus=1, auto_select_gpus=False, tpu_cores=<function _gpus_arg_default at 0x7f85d60c0a60>, log_gpu_memory=None, progress_bar_refresh_rate=1, overfit_batches=0.0, track_grad_norm=-1, check_val_every_n_epoch=1, fast_dev_run=False, accumulate_grad_batches=1, max_epochs=1, min_epochs=1, max_steps=None, min_steps=None, limit_train_batches=1.0, limit_val_batches=1.0, limit_test_batches=1.0, val_check_interval=1.0, flush_logs_every_n_steps=100, log_every_n_steps=50, accelerator=None, sync_batchnorm=False, precision=32, weights_summary='top', weights_save_path=None, num_sanity_val_steps=2, truncated_bptt_steps=None, resume_from_checkpoint=None, profiler=None, b

Validating:  36%|██████████▍                  | 158/437 [00:26<00:46,  6.05it/s]
Epoch 0:  93%|▉| 3659/3936 [20:35<01:33,  2.96it/s, loss=3.43, v_num=0, val_loss
Validating:  37%|██████████▌                  | 160/437 [00:26<00:45,  6.06it/s]
Epoch 0:  93%|▉| 3661/3936 [20:35<01:32,  2.96it/s, loss=3.43, v_num=0, val_loss
Validating:  37%|██████████▊                  | 162/437 [00:26<00:45,  6.09it/s]
Epoch 0:  93%|▉| 3663/3936 [20:35<01:32,  2.96it/s, loss=3.43, v_num=0, val_loss
Validating:  38%|██████████▉                  | 164/437 [00:27<00:44,  6.08it/s]
Epoch 0:  93%|▉| 3665/3936 [20:36<01:31,  2.96it/s, loss=3.43, v_num=0, val_loss
Validating:  38%|███████████                  | 166/437 [00:27<00:44,  6.07it/s]
Epoch 0:  93%|▉| 3667/3936 [20:36<01:30,  2.97it/s, loss=3.43, v_num=0, val_loss
Validating:  38%|███████████▏                 | 168/437 [00:27<00:44,  6.06it/s]
Epoch 0:  93%|▉| 3669/3936 [20:36<01:30,  2.97it/s, loss=3.43, v_num=0, val_loss
Validating:  39%|███████████

Validating:  81%|███████████████████████▎     | 352/437 [00:58<00:14,  6.05it/s]
Epoch 0:  98%|▉| 3853/3936 [21:07<00:27,  3.04it/s, loss=3.43, v_num=0, val_loss
Validating:  81%|███████████████████████▍     | 354/437 [00:58<00:13,  6.04it/s]
Epoch 0:  98%|▉| 3855/3936 [21:07<00:26,  3.04it/s, loss=3.43, v_num=0, val_loss
Validating:  81%|███████████████████████▌     | 356/437 [00:58<00:13,  6.06it/s]
Epoch 0:  98%|▉| 3857/3936 [21:07<00:25,  3.04it/s, loss=3.43, v_num=0, val_loss
Validating:  82%|███████████████████████▊     | 358/437 [00:59<00:13,  6.07it/s]
Epoch 0:  98%|▉| 3859/3936 [21:08<00:25,  3.04it/s, loss=3.43, v_num=0, val_loss
Validating:  82%|███████████████████████▉     | 360/437 [00:59<00:12,  6.06it/s]
Epoch 0:  98%|▉| 3861/3936 [21:08<00:24,  3.04it/s, loss=3.43, v_num=0, val_loss
Validating:  83%|████████████████████████     | 362/437 [00:59<00:12,  6.05it/s]
Epoch 0:  98%|▉| 3863/3936 [21:08<00:23,  3.04it/s, loss=3.43, v_num=0, val_loss
Validating:  83%|███████████

In [3]:
!python train.py  --gradient_clip_val 1.0 --max_epochs 1 --default_root_dir logs  --gpus 1 --batch_size 4 --num_workers 4 --max_len 256

INFO:root:Namespace(checkpoint_path=None, lr=3e-05, warmup_ratio=0.1, model_path=None, train_file='data/train.tsv', test_file='data/test.tsv', batch_size=4, max_len=256, num_workers=4, logger=True, checkpoint_callback=True, default_root_dir='logs', gradient_clip_val=1.0, process_position=0, num_nodes=1, num_processes=1, gpus=1, auto_select_gpus=False, tpu_cores=<function _gpus_arg_default at 0x7fdcab1ad9d0>, log_gpu_memory=None, progress_bar_refresh_rate=1, overfit_batches=0.0, track_grad_norm=-1, check_val_every_n_epoch=1, fast_dev_run=False, accumulate_grad_batches=1, max_epochs=1, min_epochs=1, max_steps=None, min_steps=None, limit_train_batches=1.0, limit_val_batches=1.0, limit_test_batches=1.0, val_check_interval=1.0, flush_logs_every_n_steps=100, log_every_n_steps=50, accelerator=None, sync_batchnorm=False, precision=32, weights_summary='top', weights_save_path=None, num_sanity_val_steps=2, truncated_bptt_steps=None, resume_from_checkpoint=None, profiler=None, benchmark=False, de

Epoch 0:  93%|▉| 3655/3936 [26:12<02:00,  2.32it/s, loss=2.56, v_num=2, val_loss
Validating:  36%|██████████▎                  | 156/437 [00:24<00:43,  6.43it/s]
Epoch 0:  93%|▉| 3657/3936 [26:12<01:59,  2.33it/s, loss=2.56, v_num=2, val_loss
Validating:  36%|██████████▍                  | 158/437 [00:24<00:43,  6.39it/s]
Epoch 0:  93%|▉| 3659/3936 [26:12<01:59,  2.33it/s, loss=2.56, v_num=2, val_loss
Validating:  37%|██████████▌                  | 160/437 [00:25<00:43,  6.42it/s]
Epoch 0:  93%|▉| 3661/3936 [26:13<01:58,  2.33it/s, loss=2.56, v_num=2, val_loss
Validating:  37%|██████████▊                  | 162/437 [00:25<00:42,  6.44it/s]
Epoch 0:  93%|▉| 3663/3936 [26:13<01:57,  2.33it/s, loss=2.56, v_num=2, val_loss
Validating:  38%|██████████▉                  | 164/437 [00:25<00:42,  6.48it/s]
Epoch 0:  93%|▉| 3665/3936 [26:13<01:56,  2.33it/s, loss=2.56, v_num=2, val_loss
Validating:  38%|███████████                  | 166/437 [00:25<00:42,  6.43it/s]
Epoch 0:  93%|▉| 3667/3936 [

Epoch 0:  98%|▉| 3849/3936 [26:42<00:36,  2.40it/s, loss=2.56, v_num=2, val_loss
Validating:  80%|███████████████████████▏     | 350/437 [00:54<00:13,  6.42it/s]
Epoch 0:  98%|▉| 3851/3936 [26:42<00:35,  2.40it/s, loss=2.56, v_num=2, val_loss
Validating:  81%|███████████████████████▎     | 352/437 [00:54<00:13,  6.44it/s]
Epoch 0:  98%|▉| 3853/3936 [26:42<00:34,  2.40it/s, loss=2.56, v_num=2, val_loss
Validating:  81%|███████████████████████▍     | 354/437 [00:55<00:12,  6.47it/s]
Epoch 0:  98%|▉| 3855/3936 [26:43<00:33,  2.40it/s, loss=2.56, v_num=2, val_loss
Validating:  81%|███████████████████████▌     | 356/437 [00:55<00:12,  6.49it/s]
Epoch 0:  98%|▉| 3857/3936 [26:43<00:32,  2.41it/s, loss=2.56, v_num=2, val_loss
Validating:  82%|███████████████████████▊     | 358/437 [00:55<00:12,  6.49it/s]
Epoch 0:  98%|▉| 3859/3936 [26:43<00:32,  2.41it/s, loss=2.56, v_num=2, val_loss
Validating:  82%|███████████████████████▉     | 360/437 [00:56<00:11,  6.50it/s]
Epoch 0:  98%|▉| 3861/3936 [

In [13]:
# 데이터 전처리
import re
import pandas as pd
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열 (.) 제거
    sentence = re.sub(r'[ㄱ-ㅎㅏ-ㅣ.,?!]+[/ㄱ-ㅎㅏ-ㅣ.,?!]', '', sentence) # 여러개 자음, 모음, 구두점에서 마지막 1개만 남긴다
    sentence = re.sub("[^가-힣a-z0-9-.,!?]", " ", sentence) # 지정한 문자 제외 공백으로 전환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거

    return sentence

In [94]:
train = pd.read_csv('/aiffel/aiffel/aiffelthon/KoGPT2-summarization/data/train.tsv', delimiter = '\t')

In [14]:
test = pd.read_csv('/aiffel/aiffel/aiffelthon/KoGPT2-summarization/data/test.tsv', delimiter = '\t')

In [86]:
test

,Unnamed: 0,Id,Text,Summary,Category
0,0,9effd65e-e18f-5510-8866-d6a677314c9c,"웅 , 영업팀과장님이 보내줬는데 팀장님이 해줄지 모르겠다 저번에 부산갈때도 숙소로 ...",팀장님이 출장 가서 머물 숙소를 계속해서 더 싼 데로 하게 한다고 이야기하고 있다.,일과 직업
1,1,49b9f9b7-a79d-5283-9553-95b8f156ff90,"너는 잘가라회사 선택 잘해 , 알겠어 많이 힘들구나 나도 이제 이력서쓰고 영어도 해...",이제 이력서를 쓰고 영어도 해야 한다고 해서 첫 회사를 잘 들어가라고 했다.,일과 직업
2,2,df6b6bac-648b-5d32-86a3-9b8b215e5f95,"느낌상 대통령까지는 아니고 오시면 여사님정도오시지않을까 , 그러면서 , 샘 여기있었...",느낌상 대통령까지는 아니고 오시면 여사님 정도 오시지 않을까라며 이에 대해 이야기하...,일과 직업
3,3,3043293f-7bb5-5edb-9b1f-85120e305c3d,"숨만수이 도 숨만쉬어도 100 이내 , 한달안에 일 무조건 해야대 , 아 딱한달 ,...",한 달 안에 무조건 일을 시작해서 돈을 벌어야 한다.,일과 직업
4,4,2b9b9c39-2721-50e6-af3e-e5c00138f43d,"목요일은 외근이구 금요일은 출장 , 금요일이 당진이양? , 아닝아닝 10일이 당진이...",목요일에 외근이고 금요일에 출장인데 당진은 10일에 간다.,일과 직업
...,...,...,...,...,...
1741,1741,b03b9ad0-62fa-54a5-b552-69049bd642ee,"엄마 점심식사 뭐먹었어요 , 우린 중국집 시켜먹었다 , 오오 짜장면 탕수육?이에요 ...","엄마에게 오늘 점심 뭐 먹었냐고 묻고 엄마가 중국집에서 볶음밥, 잡채밥 시켜 먹었다...",식음료
1742,1742,ec83f40a-3c01-581d-962c-80fd62a5c888,"매운거 , 담에 점심 엽떡 , 자신있음 , 엽떡 먹는 사람 , 오 저색기 또지랄이네...",엽떡 제일 매운맛을 3명에서 도전하기로 했다.,식음료
1743,1743,2b8dad32-cb6a-5d2a-89c4-06ba5f8dc8f8,"아 술마시고싶다 취할줄알았는데 , 누룩오자 누룩 군고구마막걸리 존맛임 고구마케이크 ...",군고구마막걸리가 맛있다고 하니 빨리 마셔보고 싶다고 한다.,식음료
1744,1744,bd803ef5-c166-54ad-8d96-ac73fb055b3d,"종촌동에서 뭘 주워먹어야겠다 어른들 저녁먹고 가신대 뭐사먹지 , 근처에 뭐있나요 국...",어른들이 저녁 먹고 가신다고 해서 뭘 사 먹을까 하니 근처에 국수나무가 맛있다고 한다.,식음료


In [95]:
# 전처리 적용
clean_text_val = []

for s in tqdm(train['Text']):
    clean_text_val.append(preprocess_sentence(s))

train['Text'] = clean_text_val

clean_headlines_val = []

for s in tqdm(train['Summary']):
    clean_headlines_val.append(preprocess_sentence(s))

train['Summary'] = clean_headlines_val 

100%|██████████| 13994/13994 [00:00<00:00, 94756.46it/s]


In [8]:
import yaml
import torch
import torch.nn.functional as F
#from train_ptuning import KoGPTConditionalGeneration
from train import KoGPTConditionalGeneration
from utils import generate_next_token
path = '/aiffel/aiffel/aiffelthon/KoGPT2-summarization/logs'

In [6]:
hparams_file = path+'/tb_logs/default/version_2/hparams.yaml'
with open(hparams_file) as f:
    hparams = yaml.full_load(f)

In [9]:
inf = KoGPTConditionalGeneration.load_from_checkpoint(path+'/model_chp/epoch=00-val_loss=2.405.ckpt', hparams=hparams)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [10]:
tokenizer = inf.tokenizer

In [24]:
Input_Text = list(test['Text'])
Real_Sum = list(test['Summary'])

In [49]:
# 요약 결과 만들기

from tqdm import tqdm

my_summaries=[]
tensor = []
SUMMARY = '<unused1>'
PTUNING = '<unused2>'
EOS = '</s>'

for text in tqdm(Input_Text):
    input_ids = text
    input_tokens = tokenizer.encode(PTUNING)* 10 + tokenizer.encode(text) + tokenizer.encode(SUMMARY)
    input_tensor = torch.tensor(input_tokens).unsqueeze(0)
    eos_id = tokenizer.encode(EOS)
    #my_summaries.append(input_tensor)
    #while True:
    pred = inf.model(input_tensor)
    #next_token = generate_next_token(pred.logits, temperature=1.0, top_p=0.8)
    next_token = generate_next_token(pred.logits, temperature=1.0, top_p=0.92)

    if next_token.item() == eos_id:
        break
    else:
        input_tensor = torch.cat([input_tensor, next_token.unsqueeze(0)],1)

    tensor.append(input_tensor)
    


100%|██████████| 1746/1746 [08:26<00:00,  3.45it/s]


In [63]:
for ten in tqdm(tensor):
    summary = tokenizer.decode(ten[0][0]).split('<unused1>')[-1].strip()

    my_summaries.append(summary)

100%|██████████| 1746/1746 [00:00<00:00, 89273.15it/s]


In [70]:
tokenizer.decode(ten[0])

"<unused2><unused2><unused2><unused2><unused2><unused2><unused2><unused2><unused2><unused2> ['오늘은 맛있는 게 먹고싶네요', '집에가서 먹어염....', 'ㅜㅜ ', '얼른가야먹을텐데... 드디어 배가고파졌어요?? ᄏᄏᄏᄏ', 'ᄏᄏᄏᄏᄏ 근데 아까는 아 김밥 사먹어야지 했는데 지금은 또 아니에요', '김밥먹어여,,,,']<unused1> 아"

In [71]:
Input_Text[0]

"['웅', '영업팀과장님이 보내줬는데 팀장님이 해줄지 모르겠다 저번에 부산갈때도 숙소로 엄청 싸워서', '웅 흥 4개월가는거도아니고 3일가는데 좀해주지 거 얼마한다구', '내말이... 아니 해봤자 3일 다 합해도 몇만원 차이인데 너무해 그때는 2일이었는데도 만원 더 싼데에서 자꾸 자라고... 넘 시러..', '일단']"

In [54]:
tokenizer.decode(input_tensor[0]).split('<unused1>')[-1].strip()

'아'

In [17]:
SUMMARY = '<unused1>'
PTUNING = '<unused2>'
EOS = '</s>'

In [20]:
test['Text'][0]

'웅 , 영업팀과장님이 보내줬는데 팀장님이 해줄지 모르겠다 저번에 부산갈때도 숙소로 엄청 싸워서 , 웅 흥 4개월가는거도아니고 3일가는데 좀해주지 거 얼마한다구 , 내말이 아니 해봤자 3일 다 합해도 몇만원 차이인데 너무해 그때는 2일이었는데도 만원 더 싼데에서 자꾸 자라고 넘 시러 , 일단'

In [25]:
text = """웅 , 영업팀과장님이 보내줬는데 팀장님이 해줄지 모르겠다 저번에 부산갈때도 숙소로 엄청 싸워서 , 
웅 흥 4개월가는거도아니고 3일가는데 좀해주지 거 얼마한다구 , 
내말이 아니 해봤자 3일 다 합해도 몇만원 차이인데 너무해 그때는 2일이었는데도 만원 더 싼데에서 자꾸 자라고 넘 시러 , 일단""" 

In [26]:
input_tokens = tokenizer.encode(PTUNING)* 10 + tokenizer.encode(text) + tokenizer.encode(SUMMARY)
input_tensor = torch.tensor(input_tokens).unsqueeze(0)

eos_id = tokenizer.encode(EOS)

In [27]:
input_tokens

[11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11786,
 11481,
 12564,
 8612,
 6903,
 8168,
 20337,
 11219,
 8245,
 9668,
 13717,
 8168,
 20337,
 9098,
 8239,
 8263,
 42138,
 7182,
 9265,
 18554,
 11528,
 6828,
 7312,
 7235,
 10337,
 11750,
 15546,
 10406,
 19647,
 11481,
 9051,
 8101,
 10390,
 9130,
 11179,
 9932,
 6853,
 7235,
 7965,
 7172,
 6889,
 14147,
 6824,
 9668,
 11732,
 15084,
 8263,
 9122,
 11474,
 10013,
 6919,
 11481,
 9051,
 7071,
 21323,
 9320,
 9098,
 7662,
 8159,
 14147,
 9054,
 9318,
 12704,
 9774,
 7489,
 8104,
 15106,
 11848,
 12371,
 8711,
 12858,
 7162,
 16989,
 36322,
 7235,
 9103,
 8104,
 9267,
 24365,
 7220,
 9023,
 9042,
 7019,
 18136,
 9901,
 9039,
 7397,
 11481,
 15063,
 10]

In [ ]:
while True:
    pred = inf.model(input_tensor)
    next_token = generate_next_token(pred.logits, temperature=1.0, top_p=0.8)

    if next_token.item() == eos_id:
        break
    else:
        input_tensor = torch.cat([input_tensor, next_token.unsqueeze(0)],1)

In [27]:
tokenizer.decode(input_tensor[0])

"<unused2><unused2><unused2><unused2><unused2><unused2><unused2><unused2><unused2><unused2> ['목요일은 외근이구 금요일은 출장!!!', '금요일이 당진이양?', '아닝아닝 10일이 당진이야', '그럼 금요일은 어디로가...?', '의왕! 철기연으로 가', '오홍?']<unused1> 비즈니스를 하기 위해 국내여행 심사가 많은데 그 심사에 관심이 많다."

In [28]:
tokenizer.decode(input_tensor[0]).split('<unused1>')[-1].strip()

'비즈니스를 하기 위해 국내여행 심사가 많은데 그 심사에 관심이 많다.'